In [ ]:
import argparse
import numpy as np
import os
import sys
import yaml

from lib.utils import load_graph_data

## build an autoencoder model based on DCRNN architecture
from model.pytorch.dcrnn_supervisor import DCRNNSupervisor

### traffic speed sensor LA dataset

In [ ]:
config_filename = 'data/model/embedding/config-metr-la.yaml'
with open(config_filename) as f:
    supervisor_config = yaml.load(f)

    graph_pkl_filename = supervisor_config['data'].get('graph_pkl_filename')
    sensor_ids, sensor_id_to_ind, adj_mx = load_graph_data(graph_pkl_filename)

    supervisor = DCRNNSupervisor(adj_mx=adj_mx, **supervisor_config)
    test_embeddings = supervisor.compute_embedding('test')
    val_embeddings = supervisor.compute_embedding('val')

### traffic speed sensor BAY dataset

In [ ]:
config_filename = 'data/model/embedding/config-pems-bay.yaml'
del supervisor
with open(config_filename) as f:
    supervisor_config = yaml.load(f)

    graph_pkl_filename = supervisor_config['data'].get('graph_pkl_filename')
    sensor_ids, sensor_id_to_ind, adj_mx = load_graph_data(graph_pkl_filename)

    supervisor = DCRNNSupervisor(adj_mx=adj_mx, **supervisor_config)
    bay_embeddings = supervisor.compute_embedding('test')

### air pollution PM2.5 dataset

In [ ]:
config_filename = 'data/model/embedding/pollution_pm25.yaml'
#del supervisor
with open(config_filename) as f:
    supervisor_config = yaml.load(f)

    graph_pkl_filename = supervisor_config['data'].get('graph_pkl_filename')
    sensor_ids, sensor_id_to_ind, adj_mx = load_graph_data(graph_pkl_filename)

    supervisor = DCRNNSupervisor(adj_mx=adj_mx, **supervisor_config)
    pm25_embeddings = supervisor.compute_embedding('test')

### air pollution PM10 dataset

In [ ]:
config_filename = 'data/model/embedding/pollution_pm10.yaml'
del supervisor
with open(config_filename) as f:
    supervisor_config = yaml.load(f)

    graph_pkl_filename = supervisor_config['data'].get('graph_pkl_filename')
    sensor_ids, sensor_id_to_ind, adj_mx = load_graph_data(graph_pkl_filename)

    supervisor = DCRNNSupervisor(adj_mx=adj_mx, **supervisor_config)
    pm10_embeddings = supervisor.compute_embedding('test')

### dataset embedding shape

In [9]:
print(test_embeddings.shape)
print(val_embeddings.shape)
print(bay_embeddings.shape)
print(pm25_embeddings.shape)
print(pm10_embeddings.shape)

(6912, 13248)
(3456, 13248)
(8000, 20800)
(6528, 1600)
(6528, 2368)


### k-center greedy algorithm

In [11]:
import numpy as np
from sklearn.metrics import pairwise_distances

def update_distances(features, cluster_center, min_dist, only_new=True, reset_dist=False):
    if reset_dist:
      min_dist = None
    if cluster_center:
      # Update min_distances for all examples given new cluster center.
      x = features[cluster_center]
      dist = pairwise_distances(features, x, metric='euclidean')

      if min_dist is None:
        min_dist = np.min(dist, axis=1).reshape(-1,1)
      else:
        min_dist = np.minimum(min_dist, dist)
        
    return min_dist
        
def k_center(features, k, **kwargs):
    cluster_centers = []
    cluster_delta = []
    k_delta = 0
    n_obs = features.shape[0]
    min_dist = None

    for _ in range(k):
      if min_dist is None:
        # Initialize centers with a randomly selected datapoint
        #ind = np.random.choice(np.arange(n_obs))
        ind = 0
      else:
        ind = np.argmax(min_dist)

      min_dist = update_distances(features, [ind], min_dist, only_new=True, reset_dist=False)
      cluster_centers.append(ind)
      k_delta = max(min_dist)
      cluster_delta.append(k_delta)
    print('Maximum distance from cluster centers is %0.2f'
            % k_delta)

    return cluster_centers, cluster_delta


### compute core set as a k-center greedy computation

In [ ]:
no_clusters = 100
test_cluster, test_delta = k_center(test_embeddings, no_clusters)
val_cluster, val_delta = k_center(val_embeddings, no_clusters)
bay_cluster, bay_delta = k_center(bay_embeddings, no_clusters)
pm25_cluster, pm25_delta = k_center(pm25_embeddings, no_clusters)
pm10_cluster, pm10_delta = k_center(pm10_embeddings, no_clusters)

### Gromov-Wasserstein distance algorithm

In [10]:
import scipy as sp
import numpy as np
import ot

def ot_distance(emb1, emb2):
    ## Normalize distances
    C1, C2 = emb1, emb2
    print(C1.shape, C2.shape)
    C1 = (C1 - C1.min())/C1.max()
    C2 = (C2 - C2.min())/C2.max()

    p = ot.unif(C1.shape[0])
    q = ot.unif(C2.shape[0])

    gw0, log0 = ot.gromov.gromov_wasserstein(
        C1, C2, p, q, 'square_loss', verbose=True, log=True)

    gw, log = ot.gromov.entropic_gromov_wasserstein(
        C1, C2, p, q, 'square_loss', epsilon=5e-4, log=True, verbose=True)

    print('Gromov-Wasserstein distances: ' + str(log0['gw_dist']))
    print('Entropic Gromov-Wasserstein distances: ' + str(log['gw_dist']))
    
    return log0['gw_dist'], log['gw_dist']

### compute G-W distance using the core set of each dataset

In [13]:
val_C = sp.spatial.distance.cdist(val_embeddings[val_cluster], val_embeddings[val_cluster])
test_C = sp.spatial.distance.cdist(test_embeddings[test_cluster], test_embeddings[test_cluster])
bay_C = sp.spatial.distance.cdist(bay_embeddings[bay_cluster], bay_embeddings[bay_cluster])
pm25_C = sp.spatial.distance.cdist(pm25_embeddings[pm25_cluster], pm25_embeddings[pm25_cluster])
pm10_C = sp.spatial.distance.cdist(pm10_embeddings[pm10_cluster], pm10_embeddings[pm10_cluster])

In [14]:
ot_distance(pm10_C, val_C)

(100, 100) (100, 100)
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|4.042531e-02|0.000000e+00|0.000000e+00
    1|1.765360e-02|1.289918e+00|2.277170e-02
    2|1.765360e-02|0.000000e+00|0.000000e+00
It.  |Err         
-------------------
    0|2.554424e-02|
   10|3.641329e-03|
   20|4.910716e-13|
Gromov-Wasserstein distances: 0.017653603370687873
Entropic Gromov-Wasserstein distances: 0.014928391542709167


(0.017653603370687873, 0.014928391542709167)